# AgriAdapt indicator-probability demo

For demonstration purposes only. Contact Ted Wong (ted.wong@wri.org) with questions.

In [1]:
import planetary_computer
import xarray as xr
import fsspec
import pystac_client
import numpy as np
import pandas as pd
import datetime, calendar
import folium
import ipywidgets as widgets
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles

In [6]:
#import sys
#!{sys.executable} -m pip install pip earthengine-api

In [4]:
import ee
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/home/google_cred.json')

In [5]:
#ee.Initialize()
ee.Initialize(credentials)

In [7]:
LEAPLENGTH = {'UKESM1-0-LL': 360,
 'NorESM2-MM': 365,
 'NorESM2-LM': 365,
 'MRI-ESM2-0': 366,
 'MPI-ESM1-2-LR': 366,
 'MPI-ESM1-2-HR': 366,
 'MIROC6': 366,
 'MIROC-ES2L': 366,
 'KIOST-ESM': 365,
 'KACE-1-0-G': 360,
 'IPSL-CM6A-LR': 366,
 'INM-CM5-0': 365,
 'INM-CM4-8': 365,
 'HadGEM3-GC31-MM': 360,
 'HadGEM3-GC31-LL': 360,
 'GFDL-ESM4': 365,
 'GFDL-CM4_gr2': 365,
 'GFDL-CM4': 365,
 'FGOALS-g3': 365,
 'EC-Earth3-Veg-LR': 366,
 'EC-Earth3': 366,
 'CanESM5': 365,
 'CNRM-ESM2-1': 366,
 'CNRM-CM6-1': 366,
 'CMCC-ESM2': 365,
 'CMCC-CM2-SR5': 365,
 'BCC-CSM2-MR': 365,
 'ACCESS-ESM1-5': 366,
 'ACCESS-CM2': 366,
 'TaiESM1': 365,
 'ERA5': 366
}

EXCLUDED_MODELS = ['TaiESM1', 'ERA5']    # TaiESM1 model has major known biases

MODELS = [i for i in LEAPLENGTH.keys() if not i in EXCLUDED_MODELS]

HIST_START = 1980
HIST_END = 2015

EARLY_YEAR = 2030
LATE_YEAR = 2080

INITIAL_TARGETYEAR = 2040
INITIAL_THRESHOLD = 60

INITIAL_WINDOW = (' 15 Apr ', ' 10 Sep ')

INITIAL_LATLON = (9.9, -84.0)

In [8]:
def dtg34(windowdata):
    return np.sum(windowdata >= 35, axis=1)
def dtl10(windowdata):
    return np.sum(windowdata <= 10, axis=1)
def aplx(windowdata):
    return np.sum(windowdata, axis=1)


indicators = {
    'dtg34': {
            'name': 'dtg34',
            'greaterthan': True,
            'nex_varname': 'tasmax',
            'era5_varname': 'maximum_2m_air_temperature',
            'range': [1, 365, 1],
            'nex_multiply': 1,
            'nex_add': -273.15,
            'era5_multiply': 1,
            'era5_add': -273.15,
            'function': dtg34
    },
    'dtl10': {
            'name': 'dtl10',
            'greaterthan': False,
            'nex_varname': 'tasmin',
            'era5_varname': 'minimum_2m_air_temperature',
            'range': [1, 365, 1],
            'nex_multiply': 1,
            'nex_add': -273.15,
            'era5_multiply': 1,
            'era5_add': -273.15,
            'function': dtl10
    },
    'aplx': {
            'name': 'aplx',
            'greaterthan': False,
            'nex_varname': 'pr',
            'era5_varname': 'total_precipitation',
            'range': [0, 2000, 100],
            'nex_multiply': 86400,
            'nex_add': 0,
            'era5_multiply': 1000,
            'era5_add': 0,
            'function': aplx
    }    
}

In [9]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/"
)
collection = catalog.get_collection("nasa-nex-gddp-cmip6")

In [10]:
def d2j(datestring):
    d = datetime.date.fromisoformat(datestring)
    jday = d.timetuple().tm_yday
    if calendar.isleap(d.year) and jday > 59:
        jday -= 1
    return jday
def j2d(jdate, year):
    y = year
    jan1 = datetime.date(year=year, month=1, day=1)
    newdate = jan1 + datetime.timedelta(days=jdate-1)
    return newdate.isoformat()

In [11]:
statusbox = widgets.HTML()
resultbox = widgets.HTML()
debugbox = widgets.HTML()

In [12]:
def indicator_magnitudes(lat, lon, window, year, indicator):
    geom = {
        'type': Point,
        'coordinates': [lon, lat]
    }
    nex_varname = indicator['nex_varname']
    indicator_fxn = indicator['function']
    window_start, window_end = window
    thevar = []
    for model in MODELS:
        statusbox.set_trait('value', 'Collecting model projections<br><br>{0} / {1}'.format(MODELS.index(model) + 1, len(MODELS)))
        debugbox.set_trait('value', 'Searching catalog for {}'.format(model))
        search = catalog.search(
            collections=["nasa-nex-gddp-cmip6"],
            datetime="{0}/{0}".format(year),
            query={"cmip6:model": {"eq": model}, "cmip6:scenario": {"eq": "ssp585"}},
        )
        items = search.get_all_items()
        if items:
            item = items[0]
            if nex_varname in list(item.assets.keys()):
                debugbox.set_trait('value', 'Retrieving {} from STAC'.format(model))
                signed_item = planetary_computer.sign(item)
                debugbox.set_trait('value', 'Retrieving {} from STAC (1)'.format(model))
                href = signed_item.assets[nex_varname].href
                debugbox.set_trait('value', 'Retrieving {} from STAC (1.1)'.format(model))
                openf = fsspec.open(href).open()
                debugbox.set_trait('value', 'Retrieving {} from STAC (1.2)'.format(model))
                data = xr.open_dataset(openf)
                debugbox.set_trait('value', 'Retrieving {} from STAC (2)'.format(model))
                localdata = data.sel(lat=lat, lon=lon, method='nearest')
                debugbox.set_trait('value', 'Retrieving {} from STAC (3)'.format(model))
                if LEAPLENGTH[model] > 360:
                    startj = d2j("{0}-{1}".format(year, window_start))
                    endj = d2j("{0}-{1}".format(year, window_end))
                    debugbox.set_trait('value', 'Appending {} to all-models array'.format(model))
                    thevar.append(localdata.variables[nex_varname][startj-1:endj])
    debugbox.set_trait('value', '')
    statusbox.set_trait('value', 'Processing model projections<br><br>')
    thevar_arr = (np.array(thevar, float) * indicator['nex_multiply']) + indicator['nex_add']
    return indicator_fxn(thevar_arr)

In [13]:
def model_projections(lat, lon, window, target_year, indicator):
    statusbox.set_trait('value', 'Collecting model projections')
    mags = []
    for year in range(target_year, target_year + 1):
        mags.append(indicator_magnitudes(lat, lon, window, year, indicator))
    #proj_hazard = pd.DataFrame(gtes, index=list(range(PROJ_START, PROJ_END+1)))
    return np.reshape(mags, (target_year - target_year + 1, 25))  # 25 is number of usable models

In [14]:
def hist_mags_gee(lat, lon, window, indicator):
    era5_varname = indicator['era5_varname']
    is_forecast = era5_varname == "precipitation_amount_1hour_Accumulation"
    indicator_fxn = indicator['function']
    statusbox.set_trait('value', 'Collecting historical values')
    startmonth, endmonth = datetime.datetime.strptime(window[0], '%m-%d').month, datetime.datetime.strptime(window[1], '%m-%d').month

    gee_geom = ee.Geometry.Point((lon, lat))

    allyears = []
    for year in range(HIST_START, HIST_END + 1):
        window_start = datetime.datetime.strptime('{0}-{1}'.format(year, window[0]), '%Y-%m-%d')
        window_end_plusone = datetime.datetime.strptime('{0}-{1}'.format(year, window[1]), '%Y-%m-%d') + datetime.timedelta(days=1) 
        statusbox.set_trait('value', 'Collecting historical values<br><br>{0} / {1}'.format(list(range(HIST_START, HIST_END + 1)).index(year) + 1, HIST_END - HIST_START + 1))

        era5_varname = indicator['era5_varname']
        ERA5 = ee.ImageCollection("ECMWF/ERA5/DAILY")
        era5_vars = ERA5.select(era5_varname)


        yeardata = pd.DataFrame(era5_vars.filterDate(window_start.strftime('%Y-%m-%d'), window_end_plusone.strftime('%Y-%m-%d')).getRegion(gee_geom, 25000, 'epsg:4326').getInfo())[4][1:]

        allyears.append(yeardata)
    return indicator_fxn((np.array(allyears) * indicator['era5_multiply']) + indicator['era5_add'])

In [15]:
def get_naiveprob(magnitude_data, threshold, want_greater=True):
    if want_greater:
        return np.mean(magnitude_data >= threshold)
    else:
        return np.mean(magnitude_data <= threshold)

def naives(lat, lon, thresh, window, indicator):

    statusbox.set_trait('value', 'Calculating Bayesian parameters')
    #early_mags = indicator_magnitudes(EARLY_YEAR)
    #late_mags = indicator_magnitudes(LATE_YEAR)

    early_naiveprob = get_naiveprob(indicator_magnitudes(lat, lon, window, EARLY_YEAR, indicator), thresh)
    late_naiveprob = get_naiveprob(indicator_magnitudes(lat, lon, window, LATE_YEAR, indicator), thresh)
    #early_naiveprobs = []
    #late_naiveprobs = []
    #for threshold in range(0,365):
    #    early_naiveprobs.append(get_naiveprob(early_mags, threshold, True))
    #    late_naiveprobs.append(get_naiveprob(late_mags, threshold, True))
    #early_naive = pd.DataFrame(early_naiveprobs, index=list(range(0,365)))
    #late_naive = pd.DataFrame(late_naiveprobs, index=list(range(0,365)))
    return early_naiveprob, late_naiveprob

In [16]:
def alphabeta(hist_mags, threshold, want_greater=True):
    if want_greater:
        return(1 + np.sum(hist_mags >= threshold), 1 + np.sum(hist_mags < threshold))
    else:
        return(1 + np.sum(hist_mags <= threshold), 1 + np.sum(hist_mags > threshold))
    
#alpha, beta = alphabeta(hist_hazards, threshold)

In [17]:
def do_one_location(button):
    resultbox.set_trait('value', 'Calculating...')
    #hazard = HAZARD
    mlat, mlon = marker.location
    windowstart_str = datetime.datetime.strptime(window_slider.value[0], ' %d %b ')
    windowend_str = datetime.datetime.strptime(window_slider.value[1], ' %d %b ')
    window = (windowstart_str.strftime('%m-%d'), windowend_str.strftime('%m-%d'))
    indicator = indicators[indicator_selector.value]
    threshold = threshold_slider.value
    target_year = year_slider.value
    proj_hazard = model_projections(mlat, mlon, window, target_year, indicator)
    hist_hazards = hist_mags_gee(mlat, mlon, window, indicator)
#    earlynaive, latenaive = naives(mlat, mlon, threshold, window, indicator)
    alpha, beta = alphabeta(hist_hazards, threshold)
#    alpha_prime = alpha * (target_year - EARLY_YEAR) * (latenaive - earlynaive) / (LATE_YEAR - EARLY_YEAR)
#    beta_prime = beta * (target_year - EARLY_YEAR) * (1-(latenaive) - (1-earlynaive)) / (LATE_YEAR - EARLY_YEAR)
    hits = np.mean(proj_hazard >= threshold, axis=1)
    misses = np.mean(proj_hazard < threshold, axis=1)
#    result = (alpha_prime + hits) / (beta_prime + alpha_prime + misses + hits)
    result = (alpha + hits) / (beta + alpha + misses + hits)
    resultbox.set_trait('value', str(result))
    statusbox.set_trait('value', 'READY')

In [18]:
the_map = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=INITIAL_LATLON,
    zoom=10
    )

In [19]:
marker = Marker(location=INITIAL_LATLON, draggable=True)
#marker.on_move(do_one_location)
the_map.add_layer(marker)

In [20]:
button = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Calculate indicator with selected parameters',
    icon='calculator' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(do_one_location)

In [21]:
windowslider_startdate = datetime.datetime(2030, 1, 1)
windowslider_enddate = datetime.datetime(2030, 12, 31)
windowslider_dates = pd.date_range(windowslider_startdate, windowslider_enddate, freq='D')

windowslider_options = [( windowslider_date.strftime(' %d %b ')) for windowslider_date in windowslider_dates]
windowslider_index = (0, len(windowslider_options)-1)

window_slider = widgets.SelectionRangeSlider(
    options=windowslider_options,
    index=windowslider_index,
    value=INITIAL_WINDOW,
    description='Growing season',
    orientation='horizontal',
    layout={'width': '400px'}
)

In [22]:
threshold_slider = widgets.IntSlider(
    value=INITIAL_THRESHOLD,
    min=0,
    max=365,
    step=1,
    description='threshold',
    continuous_update=False,
    disabled=False
)
year_slider = widgets.IntSlider(
    value=INITIAL_TARGETYEAR,
    min=2030,
    max=2050,
    step=1,
    description='year of interest',
    continuous_update=False,
    disabled=False
)

In [23]:
def update_thresholdrange(e):
    indicator = indicators[indicator_selector.value]
    threshold_slider.min = indicator['range'][0]
    threshold_slider.max = indicator['range'][1]
    threshold_slider.step = indicator['range'][2]
    
indicator_selector = widgets.Dropdown(
    options=[('days temp > 34', 'dtg34'), ('days temp < 10', 'dtl10'), ('annual precip < x', 'aplx')],
    value='dtg34',
    description='Indicator ',
)
indicator_selector.observe(update_thresholdrange)

In [24]:
display(the_map)

Map(center=[9.9, -84.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [25]:
input_box = widgets.VBox([widgets.HBox([widgets.VBox([indicator_selector,threshold_slider]), widgets.VBox([year_slider, window_slider])]), button])
display(input_box)

In [26]:
output_box = widgets.HBox([statusbox, resultbox, debugbox])
display(output_box)